In [4]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

# usually within local run
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)

# specific to loading UTSW data
import warnings
warnings.simplefilter('ignore')
from ptsa.data.readers import EEGReader
from ptsa.data.MatlabIO import read_single_matlab_matrix_as_numpy_structured_array
from glob import glob


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import numpy as np
import os
import sys; sys.path.append('/home1/john/johnModules')
import dill, pickle
import mne
from copy import copy
from scipy.signal import firwin,filtfilt,kaiserord
from ptsa.data.filters import ButterworthFilter
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from SWRmodule import CMLReadDFRow,get_bp_tal_struct,get_elec_regions,ptsa_to_mne
from general import superVstack
from SWRmodule import downsampleBinary,LogDFExceptionLine,getBadChannels,getStartEndArrays,getSecondRecalls,\
                    removeRepeatedRecalls,getSWRpathInfo,selectRecallType,getMTLregions,correctEEGoffset,\
                    getOutputPositions,getRecallsBeforeIntrusions,getSerialposOfRecalls,getElectrodeRanges,\
                    detectRipplesHamming,detectRipplesButter
import pingouin as pg
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)

### PARAMS ###

save_values = 1

selected_period = 'surrounding_recall' 
# 'surrounding_recall': aligned to time of free recall 
# 'whole_retrieval': aligned to beginning of retrieval period (beep_off)
# 'encoding': aligned to word_on 
# 'math': aligned to math problem on
# 'math_retrieval': aligned to math problem key-in time

# there are three periods this code is set up to look at: periods aligned to recall, the entire retrieval period, and the encoding period
recall_type_switch = 0 # how do we select recalls?? Numbers 0:3
# 0: Original analysis taking only recalls without a recall in 2 s IRI before them
# 1: Take these same recalls, but keep only those WITH a recall within 2 s after they occur
# 2: test condition where we look at second recalls within IRI ONLY
# 3: ISOLATED only!
# 4: only first recall of every retrieval period
# 5: take only those recalls that come second in retrieval period within 2 s of first retrieval
# 6: take only NOT first recall of every retrieval period
# 7: take only NOT first recall AND ISOLATED trials (this should REALLY maximize SWR bump)
# 10: same as 0 but with no IRI (mostly just to see number of recalls)

selected_region = HPC_labels #HPC_labels # ENT_labels+HPC_labels

remove_soz_ictal = 0 # 0 for nothing, 1 for remove SOZ, 2 for keep ONLY SOZ ###

min_ripple_rate = 0.1 # Hz.
max_ripple_rate = 1.5 # Hz.
max_trial_by_trial_correlation = 0.05 # if ripples correlated more than this remove them
max_electrode_by_electrode_correlation = 0.2 #???

filter_type = 'hamming' # see local version below for details 
# butter (Vaz algorithm)
# hamming (Norman algorithm)
# hamming125200 (Norman algorithm meant to capture "true" ripple frequency per Sullivan...Buzsaki 2011
# hamming140250 (Same idea, but higher bands)


## patient info ## (did this manually since there are so many inconsistencies in behavior/eeg labeling)

subs = ['UT159', 'UT162', 'UT165',
        'UT167','UT178', 'UT185',
        'UT191', 'UT195','UT199',
        'UT238'] # UT194 is unusable
# 'UT255' is new but I don't see it?
# manually assigning the actual session numbers in the behavioral folder in home/eeg/UTXXX/
session_nums = [ [1,2],[0,1],[0,1],
                 [0,1],[0,1],[0,1],
                 [0,1],[1,2],[1,2],
                 [0,1] 
               ] # note: the missing 0s are FR sessions not in blinded condition (UT159,UT195,UT199)
scopo_indicator = [ [0,1],[1,0],[0,1],
                   [0,1],[1,0],[1,0],
                   [0,1],[1,0],[0,1],
                   [1,0]
                  ] # matches session number
# the organization of the behavioral folders does not match the EEG folders so use these "experiment" names to get the EEG
behav_names = [ ['FR1_scopolamine','FR1_scopolamine'],['FR1_scopolamine','FR1_scopolamine'],['FR1_scopolamine','FR1_scopolamine'],
               ['FR1','FR1_scopolamine'],['FR1_scopolamine','FR1_scopolamine'],['FR1_scopolamine','FR1_scopolamine'],
               ['FR1_scopolamine','FR1_scopolamine'],['FR1_scopolamine','FR1_scopolamine'],['FR1_scopolamine','FR1_scopolamine'],
               ['FR1_scopolamine','FR1_scopolamine']
              ]
eeg_names = [ ['FR1_scopolamine','FR1_scopolamine'],['FR1_Scopolamine','FR1'],['FR1','FR1'],
              ['FR1','FR1'],['FR1_scop','FR1_scop'],['FR1_scop','FR1_scop'],
              ['FR1_scop','FR1_scop'],['FR1_scop','FR1_scop'],['FR1_scop','FR1_scop'],
              ['FR1_scop','FR1_scop']
            ]


## recall params
recall_minimum = 2000
IRI = 2000 # inter-ripple interval...remove ripples within this range (keep only first one and remove those after it)
retrieval_whole_time = 30000
# encoding params
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.
eeg_buffer = 300 # buffer to add to either end of IRI when processing eeg #**

soz_keep = [0,1] # 0 are good elecs and 1 are SOZ elecs. Never keep 2 (bad leads) ###
if remove_soz_ictal == 1:
    soz_keep = [0]
elif remove_soz_ictal == 2:
    soz_keep = [1]

### END PARAMS ###


if 'entorhinal' in selected_region and 'left hippocampus' in selected_region:
    region_name = 'HPC_ENT'
elif 'entorhinal' in selected_region:
    region_name = 'ENT'
elif 'left hippocampus' in selected_region:
    region_name = 'HPC'
if selected_period == 'surrounding_recall':
    psth_start = -IRI # only makes sense to look at period <= IRI
    psth_end = IRI # how long to grab data after recall
elif selected_period == 'whole_retrieval':
    psth_start = -IRI # doesn't have to be IRI just 2000 ms is convenient
    psth_end = IRI+retrieval_whole_time
elif selected_period == 'encoding':
    psth_start = pre_encoding_time
    psth_end = encoding_time
elif (selected_period == 'math') | (selected_period == 'math_retrieval'): #$$
    psth_start = -2000 # just use 2000 since math problems are actually like 5 s apart typically
    psth_end = 2000  
    
# loop across subjects and sessions    
for i_sub,sub in enumerate(subs):

    for i_session,session in enumerate(session_nums[i_sub]): # sessions numbers were manually added into a list of lists above    

        ripple_array = []; fr_array = []; 
        trial_nums = []; 
        session_ct = 0; channel_ct = 0; total_channel_ct = 0
        HPC_names = []; sub_names = []; sub_sess_names = []
        electrodes_per_session = []
        total_lists = 0; total_recalls = 0; kept_recalls = 0
        align_adjust = 0
        ent_elec_ct = []; sd_regions = []; not_sd_regions = []
        ripple_ied_accum_ct = []
        time_add_save = [];             
        encoded_word_key_array = []
        list_num_key = []

        list_recall_num_array = []; rectime_array = []; recall_before_intrusion_array = [] # new ones added 2020-11-24
        serialpos_array = [] # used to be encoding info but commandeered for surrounding_recalls ~~~
        recall_position_array = []; recall_index_array = []

        trial_by_trial_correlation = []; elec_by_elec_correlation = []
        elec_ripple_rate_array = []

        channel_coords = []; electrode_labels = []; channel_nums = []

        try:

            ### First get the recall and word events ###

            evs = pd.DataFrame.from_records( read_single_matlab_matrix_as_numpy_structured_array(
                '/data/eeg/'+sub+'/behavioral/'+behav_names[i_sub][i_session]+'/session_'+str(session)+'/events.mat', 'events') )   

            # UTSW data annoyingly uses 'item' instead of 'item_name'
            evs = evs.rename(columns={"item": "item_name"})

            # 1) change evs.recalled to False 2) comment out nonrepeat_indicator>0 for good_recalls = (or it removes all the -1s) 
            # 3) set getOutputPositions to [] 4) change save name at bottom to '_intrusions.p'
            evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.intrusion==0)] # recalled word AND correct (from this list...False for instrusions).        
            word_evs = evs[evs['type']=='WORD'] # get words 

            # select which recalls??         
            [recall_selection_name,selected_recalls_idxs] = selectRecallType(recall_type_switch,evs_free_recall,IRI,recall_minimum)

            _,nonrepeat_indicator = removeRepeatedRecalls(evs_free_recall,word_evs) # remove free recalls that happened more than once

            # let's make sure remaining recalls are not repeated, have eeg, and are not from non-practice lists
            good_recalls = selected_recalls_idxs & np.array(evs_free_recall.eegoffset>-1) & np.array(evs_free_recall.list > -1) & (nonrepeat_indicator>0)
            evs_free_recall = evs_free_recall[good_recalls]

            # get temp key of recalls that lead to intrusions ~~~
            pre_intrusion_recalls = getRecallsBeforeIntrusions(evs,evs_free_recall)

            if len(evs_free_recall)==0: #...and that any are left!
                continue

            # get output position in each list for this session's recalls
            session_corrected_list_ops = getOutputPositions(evs,evs_free_recall)


            ### Get UTSW localizations ###

            contacts = pd.DataFrame.from_records(np.recfromtxt('/data/eeg/'+sub+'/eeg.noreref/jacksheet.txt', encoding='utf-8'))
            contacts = contacts.rename(columns={'f0':'contact', 'f1':'label'})        

            with open('/data/eeg/'+sub+'/docs/'+sub+'_depth_el_info.txt', 'r') as f:
                lines = f.readlines()
            lines = [line.rstrip('\n') for line in lines]

            depth_info = pd.DataFrame(columns=['label', 'description'])
            for line in lines:
                split = line.split()
                try:
                    depth_info = depth_info.append({'label':split[1], 'description':' '.join(split[2:])}, ignore_index=True)
                except IndexError:
                    continue
            contacts = depth_info.merge(contacts, how='inner') # note it's contacts and not pairs


            ###  Load eeg ###

            if selected_period == 'surrounding_recall':
                total_recalls = total_recalls + len(evs_free_recall) # get total recalls from lists
                total_lists = total_lists + len(evs[evs.type=='WORD'].list.unique()) # get total lists
                kept_recalls = kept_recalls + len(evs_free_recall)
                eeg_events = evs_free_recall

    #             # fix EEG offset due to Unity implementation error
    #             eeg_events = correctEEGoffset(sub,session,exp,reader,eeg_events) # I dn't think UTSW has this error

            # We want to get the base file name in a clean/systematic way, probably by globbing on the experiment_session name 
            # (except that experiment names were inconsistent so I manually wrote them out above)
            root = '/data/eeg'
            paths = glob(os.path.join(root, sub, 'eeg.noreref', sub+'_'+eeg_names[i_sub][i_session]+'_'+str(session)+'*'), recursive=True)
            file_base = os.path.basename(paths[0]).split('.')[0]
            base_name = os.path.join(os.path.dirname(paths[0]), file_base)

            eeg_events['eegfile'] = base_name

            eeg_ptsa = EEGReader(eeg_events.to_records(),
                            channels= contacts.contact.values[contacts.description.str.contains('hippocampus')],
                            start_time=(psth_start-eeg_buffer+align_adjust)/1000,
                            end_time=(psth_end+eeg_buffer+align_adjust)/1000).read()
            
            # switch to event X channel X time (to match reader.load_eeg)
            eeg_ptsa = eeg_ptsa.transpose('events','channels','time')
            eeg_ptsa = eeg_ptsa.rename({'events':'event','channels':'channel'}) # to get same naming conventions from reader.load_eeg

            sr = eeg_ptsa.samplerate        
            sr = int(sr.values)
            
            # If using Vaz algo can't do >250 Hz IED detection so don't use this sub
            if (sr<990) and filter_type=='butter': #^^^
                print('Cannot use '+sub+' since using Vaz algo and sr ≤ 500!')
                continue        

            # if weird samplerate, add a few ms to make the load work
            if (499<sr<500) | (998<sr<1000):
                time_add = 1
                if (499<sr<500):
                    sr = 500
                elif (998<sr<1000):
                    sr = 1000
                while eeg.shape[2] < (psth_end-psth_start+2*eeg_buffer)/(1000/sr):
                    eeg = reader.load_eeg(events=eeg_events, rel_start=psth_start-eeg_buffer+align_adjust, 
                                          rel_stop=psth_end+eeg_buffer+time_add+align_adjust, clean=True, scheme=pairs)
                    if time_add>50: #**
                        continue
                    time_add+=1
                time_add_save.append(time_add)
                eeg.samplerate = sr # need to overwrite those that were just fixed

#             eeg_ptsa = eeg.to_ptsa() # already ptsa when using EEGReader
#             eeg = None # clear variable
#     #         break;break;break # to look at eeg_ptsa (plots at bottom of notebook) before filtering

            # line removal...don't do 120 for now
            eeg_ptsa = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[58.,62.], filt_type='stop', order=4).filter()
            eeg_ptsa = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[178.,182.], filt_type='stop', order=4).filter()

            ## FILTERS ##
            trans_width = 5. # Width of transition region, normalized so that 1 corresponds to pi radians/sample. 
            # That is, the frequency is expressed as a fraction of the Nyquist frequency.
            ntaps = (2/3)*np.log10(1/(10*(1e-3*1e-4)))*(sr/trans_width) # gives 400 with sr=500, trans=5
            # formula from Belanger's Digital Processing of Signals
            # see https://dsp.stackexchange.com/questions/31066/how-many-taps-does-an-fir-filter-need for how to use

            if sr == 512 or sr == 1024 or sr == 1023.999: # last one fixes R1221P @@
                ntaps = np.ceil(ntaps)
            FIR_bandstop = firwin(int(ntaps+1), [70.,178.], fs=sr, window='hamming',pass_zero='bandstop')
        #     FIR_bandpass = firwin(int(ntaps), [70.,180.], fs=sr, window='hamming',pass_zero='bandpass')
            bandstop_25_60 = firwin(int(ntaps+1), [20.,58.], fs=sr, window='hamming',pass_zero='bandstop') # Norman 2019 IED
        #     bandstop_60_80 = firwin(int(ntaps+1), [60.,80.], fs=sr, window='hamming',pass_zero='bandstop') # Gelinas 2016 IED
            nyquist = sr/2
            ntaps40, beta40 = kaiserord(40, trans_width/nyquist)
            kaiser_40lp_filter = firwin(ntaps40, cutoff=40, window=('kaiser', beta40), scale=False, nyq=nyquist, pass_zero='lowpass')

            # filter for ripples using filter selected above

            if filter_type == 'hamming':
                # need to subtract out to get the filtered signal since default is bandstop but want to keep it as PTSA        
                eeg_rip_band = eeg_ptsa-filtfilt(FIR_bandstop,1.,eeg_ptsa) 
                #         eeg_rip_band = filtfilt(FIR_bandpass,1.,eeg_ptsa) # can't use ptsa_to_mne this way so use eeg - bandstopped signal
                eeg_ied_band = eeg_ptsa-filtfilt(bandstop_25_60,1.,eeg_ptsa)
            elif filter_type == 'butter':
                eeg_rip_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[80.,120.], filt_type='bandpass',order=2).filter()
    #             if sr == 500: # dropped below 250 Hz because too close Nyquist
    #                 eeg_ied_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=250., filt_type='highpass',order=2).filter() 
    #             elif sr >= 1000:    
    #                 #this seems okay since large range far from Nyquist...problem is it likely misses key events
                eeg_ied_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[250.,490.], filt_type='bandpass',order=2).filter() #^^^
                eeg_raw = ptsa_to_mne(eeg_ptsa,[0,psth_end-psth_start+2*eeg_buffer])    #**  
    #         eeg_ptsa = None # clear variable # no reason to do this in local version...really for cluster

            eeg_rip_band = ptsa_to_mne(eeg_rip_band,[0,psth_end-psth_start+2*eeg_buffer]) #** 
            _ = eeg_rip_band.apply_hilbert(envelope=True)
            eeg_ied_band = ptsa_to_mne(eeg_ied_band,[0,psth_end-psth_start+2*eeg_buffer]) #** 
            _ = eeg_ied_band.apply_hilbert(envelope=True)

            
            # I don't know if UTSW has bad_leads
            
#             # get bad channel mask
#             try:
#                 elec_cats = reader.load('electrode_categories') # this is cool
#             except:
#                 if remove_soz_ictal == 1: # if no electrode_categories file then don't consisder this patient
#                     e = 'No electrode categories for '+sub+', session '+str(session)
#                     LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt')
#                     continue
#                 else:
#                     elec_cats = []

#             bad_bp_mask = getBadChannels(tal_struct,elec_cats,remove_soz_ictal)

#             # this was specific to FR1/catFR1 sessions I went through manually
#             electrode_search_range = getElectrodeRanges(elec_regions,exp,sub,session,mont)

            region_electrode_ct = 0
            total_electrode_ct = 0
            session_ripple_rate_by_elec = []
#             for channel in electrode_search_range:
            for channel in range(np.shape(eeg_ptsa)[1]): # every channel in hippocampus due to how contacts is selected above

#                 if (elec_regions[channel] in selected_region) & \
#                 (bad_bp_mask[channel] in soz_keep): # selecting for SOZ or not ###

                total_channel_ct+=1 # total channels before artifact removal

                # get data from MNE container 
                eeg_rip = eeg_rip_band.get_data()[:,channel,:]     
                eeg_ied = eeg_ied_band.get_data()[:,channel,:]

                # select detection algorithm (note that iedlogic is same for both so always run that)

                if filter_type == 'hamming':
                    # filter IEDs 
                    eeg_ied = eeg_ied**2 # already rectified now square
                    eeg_ied = filtfilt(kaiser_40lp_filter,1.,eeg_ied) # low pass filter  
                    mean1 = np.mean(eeg_ied)
                    std1 = np.std(eeg_ied)
                    iedlogic = eeg_ied>=mean1+4*std1 # Norman et al 2018            
                    # detect ripples
                    ripplelogic = detectRipplesHamming(eeg_rip,trans_width,sr,iedlogic)
                elif filter_type == 'butter':
                    eeg_mne = eeg_raw.get_data()[:,channel,:]
                    # detect ripples
                    ripplelogic, iedlogic = detectRipplesButter(eeg_rip,eeg_ied,eeg_mne,sr)             

#                 ripple_ied_accum_ct.append([sub,session,sr,sum(ripplelogic),sum(iedlogic),sum(ripplelogic2),sum(iedlogic2),
#                                             sum(ripplelogic_enc),sum(iedlogic_enc)])

                if filter_type == 'butter': # ^^^
                    desired_sample_rate = 1000 # for Vaz algo
                if sr>desired_sample_rate: # downsampling here for anything greater than 500 (hamming) or 1000 (butter)
                    ripplelogic = downsampleBinary(ripplelogic,sr/desired_sample_rate)

                # ripples are detected, so can remove buffers now #**
                ripplelogic = ripplelogic[:,int(eeg_buffer/(1000/desired_sample_rate)):
                                          int((psth_end-psth_start+eeg_buffer)/(1000/desired_sample_rate))]

                # skip this electrode if the ripple rate is below threshold
                temp_start_array,_ = getStartEndArrays(ripplelogic)
                elec_ripple_rate = sum(temp_start_array)/temp_start_array.shape[0]/((psth_end-psth_start)/1000)
                if elec_ripple_rate < min_ripple_rate:
                    print(sub+', '+str(session)+' skipped b/c below ripple rate thresh for ch.: '+str(channel))
                    continue
                elif elec_ripple_rate > max_ripple_rate:
                    print(sub+', '+str(session)+' skipped b/c ABOVE ripple rate thresh for ch.: '+str(channel))
                    continue # skip this electrode

                # check the ripples for this electrode and make sure they're not super correlated across trials

                # first, bin the array so can get more realistic correlation not dependent on ms timing
                binned_ripplelogic = downsampleBinary(ripplelogic,10) # downsample by 10x so 10 ms bins
                trial_ripple_df = pd.DataFrame(data=np.transpose(binned_ripplelogic))
                num_cols = len(list(trial_ripple_df))
                trial_ripple_df.columns = ['col_' + str(i) for i in range(num_cols)] # generate range of ints for suffixes
                temp_tbt_corr = np.mean(pg.pairwise_corr(trial_ripple_df,method='spearman').r)
                if temp_tbt_corr > max_trial_by_trial_correlation:
                    print(sub+', '+str(session)+' skipped b/c above trial-by-trial correlation for ch.: '+str(channel))
                    continue

                ## if this electrode passes SAVE data ##

                trial_by_trial_correlation.append(temp_tbt_corr) # corr b/w trials
                elec_ripple_rate_array.append(elec_ripple_rate) # record the average ripple rate for this electrode

                # append arrays across electrodes
                ripple_array = superVstack(ripple_array,ripplelogic) # append across electrodes
                session_ripple_rate_by_elec = superVstack(session_ripple_rate_by_elec,np.mean(binned_ripplelogic,0)) # for correlation b/w elecs below

                if selected_period == 'encoding':
                    encoded_word_key_array.append(encoded_word_key) # save the key for each electrode so easier to unpack later
                    serialpos_array.append(evs_encoding_words.serialpos)
                    if len(encoded_word_key) != ripplelogic.shape[0]:
                        e = 'Encoded word key and ripple_array dont match for '+sub+str(session)+'_'+str(channel)
                        LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt')

                elif selected_period == 'whole_retrieval':
                    # make a matrix of the times of free recall if looking at whole retrieval period
                    list_nums = evs_rets.list.unique() # count all list numbers for this session
                    temp_fr_array = np.zeros((len(list_nums),retrieval_whole_time))
                    for ln in list_nums:
                        list_times = evs_free_recall[evs_free_recall.list==ln].rectime # I confirmed these are accurate relative to REC_START
                        temp_fr_array[ln-list_nums[0]][np.array(list_times)] = 1 # subtract from list_nums[0] to recenter fr_array on 0 trial           
                    fr_array = superVstack(fr_array,temp_fr_array)

                elif selected_period == 'surrounding_recall':    # ~~~                                
                    # adding new values 2020-11-24 for some suggested analyses from group

                    # key of serialpos for recalls
                    list_nums = evs_free_recall.list.unique()   
                    temp_recalls_serialpos = []
                    for ln in list_nums:
                        temp_sp = getSerialposOfRecalls(evs_free_recall,word_evs,ln)
                        temp_recalls_serialpos.extend(temp_sp)
                        # recall number per list
                        list_recall_num_array.extend(np.tile(len(temp_sp),len(temp_sp)))
                        # recall position per list
                        recall_position_array.extend(np.arange(len(temp_sp))+1) # 1-indexed
                    serialpos_array.extend(np.array(temp_recalls_serialpos))

                    # key of rectimes for recalls
                    rectime_array.extend(np.array(evs_free_recall.rectime)) 

                    # key of recall index numbers so I can use SWR times from start_array to grab SWR-locked EEG
                    recall_index_array.extend(np.array(evs_free_recall.index))

                    # key of recalls before intrusions
                    recall_before_intrusion_array.extend(pre_intrusion_recalls)

                elif (selected_period == 'math') | (selected_period == 'math_retrieval'):

                    list_nums = eeg_events.list.unique()  
                    for ln in list_nums:
                        num_corr_math = sum(eeg_events.list==ln)
                        list_recall_num_array.extend(np.tile(num_corr_math,num_corr_math))
                        recall_position_array.extend(np.arange(num_corr_math+1)) # correct math position

                    # either way save the rectime array
                    rectime_array.extend(np.array(eeg_events.rectime)) 
                    encoded_word_key_array.extend(np.array(eeg_events.test)) # record the actual math problem  

                channel_ct+=1 # total channels from beginning 
                region_electrode_ct+=1 # channel ct for this session
#                 HPC_names.append(elec_regions[channel])
                sub_names.append(sub)
                sub_sess_names.append(sub+'-'+str(session))
                print('Cumulative array size after channel: '+str(channel))
                print(ripple_array.shape)
                trial_nums = np.append(trial_nums,ripplelogic.shape[0])
                electrode_labels.append(contacts.iloc[channel].label) # get names of electrodes so can look for them across sessions
                channel_nums.append(channel)
                list_num_key.extend(eeg_events.list)

                # get atlas coordinates for this electrode
                if 'avg.x' in contacts:
                    temp_coord = np.append(pairs.iloc[channel]['avg.x'],np.append(pairs.iloc[channel]['avg.y'],pairs.iloc[channel]['avg.z']))
                elif 'ind.x' in contacts:
                    temp_coord = np.append(pairs.iloc[channel]['ind.x'],np.append(pairs.iloc[channel]['ind.y'],pairs.iloc[channel]['ind.z']))
                else:
                    temp_coord = np.empty(3); temp_coord[:] = np.nan
                channel_coords.append(temp_coord)

                # additional check to make sure eeg was recorded for all events
                if selected_period == 'surrounding_recall':
                    if evs_free_recall.shape[0] != ripplelogic.shape[0]:
                        # this happens because eeg wasn't turned on. remove eegoffset=-1 trials; I DO THIS ABOVE NOW
                        good_evs_free_recall = evs_free_recall.eegoffset > -1
                        session_corrected_list_ops = session_corrected_list_ops[good_evs_free_recall]
                        e = sub+str(session)+'_still unrecorded eegs'
                        LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt')
#                 ordinal_position_array.extend(session_corrected_list_ops) # update for each electrode

            # before we keep all the electrodes in this session, remove sessions with elecs that are highly correlated.
            # this really only makes sense for the cluster version, since I only do a session at a time here
            if np.sum(session_ripple_rate_by_elec)>0 and region_electrode_ct>1:
                session_ripple_df = pd.DataFrame(data=np.transpose(session_ripple_rate_by_elec))
                num_cols = len(list(session_ripple_df))
                session_ripple_df.columns = ['col_' + str(i) for i in range(num_cols)] # generate range of ints for suffixes
    #             print(pg.pairwise_corr(session_ripple_df,method='spearman').r) # print the elec-by-elec correlations
                elec_by_elec_correlation = np.mean(pg.pairwise_corr(session_ripple_df,method='spearman').r) # correlation b/w elecs
                if elec_by_elec_correlation > max_electrode_by_electrode_correlation:
                    print('Correlation b/w electrodes too high, corr = '+str(elec_by_elec_correlation))
                else:
                    print('good set of electrodes (not correlated with each other)')

            electrodes_per_session = np.append(electrodes_per_session,region_electrode_ct)
            session_ct+=1


        except Exception as e:
            LogDFExceptionLine(row, e, 'LocalSWR_get_eeg_log.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
            continue
                
             
        ### save separately for each subject + session ###    
                
        if save_values == 1:

            # get strings for path name for save and loading cluster data
            soz_label,recall_selection_name,subfolder = getSWRpathInfo(remove_soz_ictal,recall_type_switch,selected_period,recall_minimum)

            path_name = '/scratch/john/SWR_scopolamine/'+subfolder
            if os.path.isdir(path_name) == False:
                os.mkdir(path_name)
            # hard code experiment as 'FR1_scopolamine' since there are too many changes in nomenclature...will just use manual indexing for scopo sessions
            fn = os.path.join(path_name, 
                'SWR_'+'FR1_scopolamine'+'_'+sub+'_'+str(session)+'_'+region_name+'_'+selected_period+recall_selection_name+
                              '_'+soz_label+'_'+filter_type+'.p')     # +'_no_param_removal.p')   #+'.p')  #+'.intrusions.p') #'.-wrong.p')
            with open(fn,'wb') as f:
                pickle.dump({'region_electrode_ct':region_electrode_ct,'total_electrode_ct':total_electrode_ct, 
                            'HPC_names':HPC_names, 'sub_sess_names':sub_sess_names,
                            'ripple_array':ripple_array, 'time_add_save':time_add_save,
                            'trial_nums':trial_nums,'list_num_key':list_num_key,
                            'encoded_word_key_array':encoded_word_key_array,
                            'serialpos_array':serialpos_array,'list_recall_num_array':list_recall_num_array, # ~~
                            'rectime_array':rectime_array,'recall_before_intrusion_array':recall_before_intrusion_array,
                            'recall_position_array':recall_position_array,'recall_index_array':recall_index_array, # ~~
                            'fr_array':fr_array, 'sub_names':sub_names,
                            'total_recalls':total_recalls, 'kept_recalls':kept_recalls,
                            'trial_by_trial_correlation':trial_by_trial_correlation, # one value for each electrode for this session
                            'elec_by_elec_correlation':elec_by_elec_correlation,
                            'elec_ripple_rate_array':elec_ripple_rate_array,
                            'electrode_labels':electrode_labels,'channel_coords':channel_coords,'channel_nums':channel_nums}, f)
                

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Cumulative array size after channel: 0
(39, 2000)
Cumulative array size after channel: 1
(78, 2000)
Cumulative array size after channel: 2
(117, 2000)
Cumulative array size after channel: 3
(156, 2000)
Cumulative array size after channel: 4
(195, 2000)
Cumulative array size after channel: 5
(234, 2000)
Cumulative array size after channel: 6
(273, 2000)
Cumulative array size after channel: 7
(312, 2000)
good set of electrodes (not correlated with each other)
Cumulative array size after channel: 0
(48, 2000)
Cumulative array size after channel: 1
(96, 2000)
Cumulative array size after channel: 2
(144, 2000)
Cumulative array size after channel: 3
(192, 2000)
Cumulative array size after channel: 4
(240, 2000)
Cumulative array size after channel: 5
(288, 2000)
Cumulative array size after channel: 6
(336, 2000)
Cumulative array size after channel: 7
(384, 2000)
good set of electrodes (not correlated with 

NameError: name 'row' is not defined

In [9]:
os.path.join(root, sub, 'eeg.noreref', sub+'_'+experiment+'_'+str(session)+'*')

'/data/eeg/UT162/eeg.noreref/UT162_FR1_scopolamine_0*'